#download

In [ ]:
!pip install gensim
!pip install nltk

from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
from gensim.models import Word2Vec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

--2023-05-20 06:21:20--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-05-20 06:21:21--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove.6B.zip.1      100%[===================>] 822.24M  5.13MB/s    in 2m 39s  

2023-05-20 06:24:01 (5.17 MB/s) - ‘glove.6B.zip.1’ saved [862182613/862182613]



In [ ]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
replace glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install keras-preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# train

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_recall_curve

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
lemma = WordNetLemmatizer()

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Bidirectional, LSTM, Dropout, BatchNormalization
from keras.layers import Embedding

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
embeddings_index = dict()
f = open('/content/glove.6B.50d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [ ]:
!git clone https://github.com/Ankit152/IMDB-sentiment-analysis.git
train = pd.read_csv('/content/IMDB-sentiment-analysis/IMDB-Dataset.csv')


fatal: destination path 'IMDB-sentiment-analysis' already exists and is not an empty directory.


In [ ]:
train.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
missing_val = pd.DataFrame(train.isnull().sum())
missing_val = missing_val.reset_index()
missing_val

,index,0
0,review,0
1,sentiment,0


In [ ]:
train[train['review'].isna()]

,review,sentiment


In [ ]:
train.dropna(inplace=True)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [ ]:
train.describe().T

,count,unique,top,freq
review,50000,49582,Loved today's show!!! It was a variety and not...,5
sentiment,50000,2,positive,25000


In [ ]:
train['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [ ]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')


In [ ]:
X_train = train.loc[:,'review'].values
y_train = train.loc[:,['sentiment']].values


In [ ]:
title_df=pd.DataFrame()
title_df['X_train']=X_train
title_df['y_train']=y_train

In [ ]:
from nltk.corpus import stopwords
def preprocess_text(texts):
    texts = texts.lower() 
    texts = re.sub(r'[^\x00-\x7F]+',' ', texts) 
    splitwords = texts.split()
    splitwords = filter(lambda x: x[0]!= '@' , texts.split()) 
    splitwords = [word for word in splitwords if word not in set(stopwords.words('english'))] 
    texts = " ".join(splitwords)
    return texts


In [ ]:
title_df['X_train'] = title_df.X_train.apply(preprocess_text)
display(title_df.head())

,X_train,y_train
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production. <br /><br />the f...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there's family little boy (jake) thi...,negative
4,"petter mattei's ""love time money"" visually stu...",positive


In [ ]:
#Creating Embeddings for the titles

max_len_title = title_df.X_train.apply(lambda x: len(x.split())).max()

tok_title = Tokenizer()
tok_title.fit_on_texts(title_df.X_train)
vocab_size_title = len(tok_title.word_index) + 1
encoded_title = tok_title.texts_to_sequences(title_df.X_train)
padded_title = pad_sequences(encoded_title, maxlen=max_len_title, padding='post')

vocab_size_title = len(tok_title.word_index) + 1

title_embedding_matrix = np.zeros((vocab_size_title, 50))
for word, i in tok_title.word_index.items():
    t_embedding_vector = embeddings_index.get(word)
    if t_embedding_vector is not None:
        title_embedding_matrix[i] = t_embedding_vector

In [ ]:
x_train, x_valid, Y_train, y_valid = train_test_split(padded_title, y_train, shuffle = True, test_size = 0.15)

In [ ]:
import math
from math import exp
from keras import backend as K

In [ ]:
def mod_tanh(x):
    return K.tanh(0.6*x)

In [ ]:
print(x_train)
print(Y_train)

[[   251    190    288 ...      0      0      0]
 [    53      3   7149 ...      0      0      0]
 [  4145    111     16 ...      0      0      0]
 ...
 [   244     35    904 ...      0      0      0]
 [   251    640 119821 ...      0      0      0]
 [    16    538     61 ...      0      0      0]]
[['negative']
 ['positive']
 ['positive']
 ...
 ['positive']
 ['negative']
 ['positive']]


In [ ]:
# Model for title
title_model = Sequential()
title_model.add(Embedding(vocab_size_title, 50, input_length=max_len_title, weights=[title_embedding_matrix], trainable=True))
title_model.add(Bidirectional(LSTM(20, return_sequences=True)))
title_model.add(Dropout(0.3))
title_model.add(BatchNormalization())
title_model.add(Bidirectional(LSTM(20, return_sequences=True)))
title_model.add(Dropout(0.3))
title_model.add(BatchNormalization())
title_model.add(Bidirectional(LSTM(20)))
title_model.add(Dropout(0.3))
title_model.add(BatchNormalization())
title_model.add(Dense(64, activation='relu'))
title_model.add(Dense(64, activation='relu'))
title_model.add(Dense(1, activation=mod_tanh))
title_model.compile(loss='mse', optimizer='adam', metrics=['mse', 'mae'])

In [ ]:
title_model.fit(x_train, Y_train, epochs = 10)

Epoch 1/10


UnimplementedError: ignored

# test

In [ ]:
title_valid_pred = title_model.predict(x_valid)

In [ ]:
from sklearn.metrics import mean_absolute_error
mae_title=mean_absolute_error(y_valid,title_valid_pred)


In [ ]:
score=1-((0.4*mae_title)+(0.6*mae_headline))

In [ ]:
print("Score = {} \nScore(out of 100%) = {}%".format(score,round(score*100, 2)))

In [ ]:
print(y_valid, title_valid_pred)

In [ ]:
!pip install unidecode
from unidecode import unidecode
unidecode("Ceñía")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 7.8 MB/s eta 0:00:00


'Cenia'

# Extra

In [ ]:
import numpy as np

# Get the interactive Tools for Matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.decomposition import PCA

from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec


In [ ]:
glove_file = datapath('/content/glove.6B.50d.txt')
word2vec_glove_file = get_tmpfile("glove.6B.50d.word2vec.txt")
glove2word2vec(glove_file, word2vec_glove_file)

<ipython-input-13-dfae7b24fe45>:3: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file, word2vec_glove_file)


(400000, 50)

In [ ]:
model = KeyedVectors.load_word2vec_format(word2vec_glove_file)

In [ ]:
model.most_similar('obama')

[('barack', 0.9674172401428223),
 ('bush', 0.9642481207847595),
 ('clinton', 0.9606045484542847),
 ('mccain', 0.9122934937477112),
 ('dole', 0.8878742456436157),
 ('gore', 0.884803831577301),
 ('hillary', 0.8776551485061646),
 ('rodham', 0.8401790261268616),
 ('kerry', 0.8261427879333496),
 ('biden', 0.8095825910568237)]

In [ ]:
model.most_similar(negative='banana')

[('silberbauer', 0.7262376546859741),
 ('waddington', 0.6286618709564209),
 ('shalem', 0.6120830178260803),
 ('haukeland', 0.5962643027305603),
 ('n.y.u.', 0.5937072038650513),
 ('fiddes', 0.593668520450592),
 ('kühnen', 0.5929498076438904),
 ('3450', 0.5929349064826965),
 ('academicals', 0.590404748916626),
 ('manston', 0.5903598666191101)]

In [ ]:
result = model.most_similar(positive=['woman', 'king'], negative=['man'])
print("{}: {:.4f}".format(*result[0]))

queen: 0.8524


In [ ]:
def analogy(x1, x2, y1):
    result = model.most_similar(positive=[y1, x2], negative=[x1])
    return result[0][0]

In [ ]:
analogy('japan', 'japanese', 'australia')

'australian'

In [ ]:
analogy('australia', 'beer', 'france')

'champagne'

In [ ]:
print(model.doesnt_match("breakfast cereal dinner lunch".split()))

cereal
